## HRA-API Documentation

### Comments
 - Andi added emojis to headings and INDEX below where &#x2705; = success, &#x274C; = issue, &#x2753; = uncertain if outcome is expected. 
 - An error log is [here](README.md).
 - Andi encountered `SSLVerificationError`s when running this notebook on Mac OS.

### INDEX
1. [Installation](#install) &#x2705;
2. [Imports](#imports) &#x2705;
3. [Configuration](#config) &#x2705;
4. [Methods](#methods)
    1. [Database Status](#db_status) &#x2705;
    2. [Run a Sparql query](#sparql_get) &#x2753;
    3. [Run a Sparql query (POST)](#sparql_post) &#x2753;
    4. [Get aggregate results / statistics](#agg_results) &#x2705;
    5. [Given a SpatialEntity already placed relative to a reference SpatialEntity retrieve a new direct SpatialPlacement to the given SpatialEntity IRI](#spatial_entity) &#x274C;
    6. [Get all hubmap rui locations](#rui_loc) &#x2705;
    7. [Get number of ontology term occurrences for a search](#ontology_term) &#x2705;
    8. [Get number of cell type term occurrences for a search](#cell_term_occ) &#x2705;
    9.  [Get ontology term tree nodes](#ontology_nodes) &#x2705;
    10. [Get cell type tree nodes](#cell_nodes) &#x2705;
    11. [Get tissue provider names (for filtering)](#providers) &#x2705;
    12. [Get all reference organs](#ref_organs) &#x2705;
    13. [Get all nodes to form the 3D scene for an organ](#3d_scene) &#x2705;
    14. [Get all nodes to form the 3D scene of reference body, organs, and tissues](#nodes_3d) &#x2705;
    15. [Get technology names (for filtering)](#tech) &#x2705;
    16. [Get Tissue Block Results](#tissue_block) &#x2705;
    17. [Get all GTEx rui locations (if enabled)](#gtex_rui) &#x2705;
    18. [Token](#token_usage) &#x2705;


<a id='install'></a>
### Installation &#x2705;

#### hra-api-client can be installed via `pip` from [PyPI](https://pypi.org/project/hra-api-client/#description).
***
> `pip install hra-api-client`
***

<a id='imports'></a>
### Imports &#x2705;

We import the hra-api-client as follows:

In [1]:
import hra_api_client

Although we can use hra-api-client for all our tasks, it is easier if we have individual imports as follows:

In [2]:
from hra_api_client.api import v1_api as default_api
from hra_api_client.models.aggregate_count import AggregateCount
from hra_api_client.models.database_status import DatabaseStatus
from hra_api_client.models.error_message import ErrorMessage
from hra_api_client.models.flat_spatial_placement import FlatSpatialPlacement
from hra_api_client.models.get_spatial_placement_request import GetSpatialPlacementRequest
from hra_api_client.models.min_max import MinMax
from hra_api_client.models.ontology_tree import OntologyTree
from hra_api_client.models.sparql_query_request import SparqlQueryRequest
from hra_api_client.models.spatial_entity import SpatialEntity
from hra_api_client.models.spatial_scene_node import SpatialSceneNode
from hra_api_client.models.spatial_search import SpatialSearch
from hra_api_client.models.tissue_block import TissueBlock

For the purposes of this documentation we need some other libraries imported here: 

In [3]:
import time
from pprint import pprint

<a id='config'></a>
### Configuration &#x2705;
ccf-openapi can fetch data from 
> HRA-API: https://apps.humanatlas.io/hra-api/v1 <br> 
> Staging Server: https://apps.humanatlas.io/hra-api--staging/v1 <br>
> You could also use any other instance that you decide to host our code on

You can get more information about our APIs [here](https://apps.humanatlas.io/hra-api/#/).

You'll need to point the host in the configuration to one of the links: 

In [4]:
# may need to remove `v1`
configuration = hra_api_client.Configuration(
    host = "https://apps.humanatlas.io/api" 
)

<a id='api'></a>
### Creating the API Instance &#x2705;
We will need to create an instance of ApiClient from hra-api-client. The configuration we created above will be passed as a parameter for the constructor. <br> <br>
Using the instance of ApiClient we create an instance of DefaultApi from the default_api module we imported above. This instance represents the API interface and provides methods to interact with the API endpoints defined in the OpenAPI specification.

In [5]:
api_client = hra_api_client.ApiClient(configuration)

api_instance = default_api.V1Api(api_client)

<a id='methods'></a>
### Methods

hra-api-client provides multiple methods which we have described below. Each method is described in more detail below : 

***
<a id='db_status'></a>
#### Database Status &#x2705;
On first run the database might take some time to be ready, so we run it on loop and wait until it's ready.
<pre>
Accepted Parameters:
    token - (string) (optional)
</pre>
***
> You can still use the other methods without waiting for the database to be ready, but the initial requests might take a little longer.

In [6]:
# replace dict notation with dot notation
db_ready = False
result = None
while not db_ready:
    result = api_instance.db_status()
    print(result)
    if result.status == 'Ready':
        db_ready = True
    else:
        print('Database not ready yet! Retrying...', result)
        time.sleep(2)
print('Database ready!\n', result)

status='Ready' checkback=3600000 load_time=22594 message='Database successfully loaded'
Database ready!
 status='Ready' checkback=3600000 load_time=22594 message='Database successfully loaded'


***
<a id='sparql_get'></a>
#### Run a SPARQL query &#x2753;
<pre>
Accepted Parameters: 
    token - (string) (optional)
    format - (string) (optional) (use only application/json in format, any other formats will result in errors.)
    query - (string) (required)
</pre>

In [26]:
try:
    api_response = api_instance.sparql(query='SELECT * WHERE { ?sub ?pred ?obj . } LIMIT 10', format='application/json')
    # help(api_instance.sparql)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

['head', 'results']


***
<a id='sparql_post'></a>
#### Run a SPARQL query (POST) &#x2753;
<pre>
Accepted Parameters:
    token - (string) (optional)
    format - (string) (optional) (use only application/json in format, any other formats will result in errors.)
    sparql_query_request - { "query": "" } (required)
</pre>

In [29]:
request = {"query": "SELECT * WHERE { ?sub ?pred ?obj . } LIMIT 10"}
# use only application/json in format, any other formats will result in errors.
try:
    # help(api_instance.sparql_post)
    api_response = api_instance.sparql_post(sparql_query_request=request, format='application/json')
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->sparql_post: %s\n" % e)

['head', 'results']


***
<a id='agg_results'></a>
#### Get aggregate results / statistics &#x2705;
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional) 
</pre>

In [9]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.aggregate_results(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

[AggregateCount(label='Tissue Data Providers', count=1),
 AggregateCount(label='Donors', count=2),
 AggregateCount(label='Tissue Blocks', count=5),
 AggregateCount(label='Extraction Sites', count=5),
 AggregateCount(label='Tissue Sections', count=5),
 AggregateCount(label='Tissue Datasets', count=5)]


***
<a id='spatial_entity'></a>
#### Given a SpatialEntity already placed relative to a reference SpatialEntity retrieve a new direct SpatialPlacement to the given SpatialEntity IRI &#x274C;
More Info : https://apps.humanatlas.io/hra-api/#/operations/get-spatial-placement
<pre>
Accepted Parameters:
    get_spatial_placement_request = {
        "target_iri": (string) (required)
        "rui_location": {
            A Spatial Entity defines a bounded cartesian space and the units it is measured in.
            It typically represents a real world thing, e.g., a human body, a human kidney, a tissue section,
            or an individual cell.
            
            "@id": string<uri> (required)
            "@type": (string) (required) (allowed value: SpatialEntity)
            
            "entityId": string<uri> (optional) (Identifier of the entity the spatial object represents)
            "ccf_annotations": array[string]<uri> (optional) (Annotations attached to the entity.)
            "representation_of": string<uri> (optional) (Identifier for the item the spatial object represents.)
            "reference_organ": string<uri> (optional) (Identifier for the organ the spatial object represents.)
            
            "creator": (string) (optional)
            "creator_first_name": (string) (optional)
            "creator_last_name": (string) (optional)
            "creator_orcid": (string) (optional)
            "creation_date": (string)<date>,
            
            "x_dimension": (number) (required),
            "y_dimension": (number) (required),
            "z_dimension": (number) (required),
            "dimension_units": (string) (required) (default: millimeters),
            
            "label": (string) (optional)
            "comment": (string) (optional)
            "rui_rank": (integer) (optional)
            "sex": (string) (optional)
            "side": (string) (optional) (Used for organs that come in pairs to distinguish between them, i.e. kidneys, etc)
            "slice_count": (integer) (optional)
            "slice_thickness": (number) (optional)
            
            
            "placement": {
                "@id": (string) (required),
                "@type": (string) (required) (allowed value: SpatialEntity),
                "placement_date": string<date>,
                "x_translation": (number) (required),
                "y_translation": (number) (required),
                "z_translation": (number) (required),
                "translation_units": (number) (required) (allowed value: millimeters),
                "x_rotation": (number) (required),
                "y_rotation": (number) (required),
                "z_rotation": (number) (required),
                "rotation_units": (string) (required) (allowed value: degree),
                "x_scaling": (number) (required),
                "y_scaling": (number) (required),
                "z_scaling": (number)  (optional),
                "scaling_units": (string) (required) (allowed value: degree),
                "target": (string) (required),
        }
            
        }
    }
</pre>

In [10]:
get_spatial_placement_request = {
    "target_iri": "http://purl.org/ccf/latest/ccf.owl#VHFemale",
    "rui_location": {
        "@id": "http://purl.org/ccf/1.5/f7130161-ad0a-4801-b5a4-f6297e0f1869",
        "@type": "SpatialEntity",
        "creator": "Bruce Herr",
        "creator_first_name": "Bruce",
        "creator_last_name": "Herr",
        "creation_date": "2022-04-27",
        "ccf_annotations": ["http://purl.obolibrary.org/obo/UBERON_0002015", "http://purl.obolibrary.org/obo/UBERON_0000362", "http://purl.obolibrary.org/obo/UBERON_0004200", "http://purl.obolibrary.org/obo/UBERON_0001225", "http://purl.obolibrary.org/obo/UBERON_0001284", "http://purl.obolibrary.org/obo/UBERON_0002189"],
        "x_dimension": 11,
        "y_dimension": 12,
        "z_dimension": 13,
        "dimension_units": "millimeter",
        "placement": {
            "@id": "http://purl.org/ccf/1.5/f7130161-ad0a-4801-b5a4-f6297e0f1869_placement",
            "@type": "SpatialPlacement",
            "target": "http://purl.org/ccf/latest/ccf.owl#VHFLeftKidneyV1.1",
            "placement_date": "2022-04-27",
            "x_scaling": 1,
            "y_scaling": 1,
            "z_scaling": 1,
            "scaling_units": "ratio",
            "x_rotation": -61,
            "y_rotation": 50,
            "z_rotation": -84,
            "rotation_order": "XYZ",
            "rotation_units": "degree",
            "x_translation": 20.149,
            "y_translation": 109.963,
            "z_translation": 38.79,
            "translation_units": "millimeter"
        }
    }
}
try:
    # Get spatial placement
    api_response = api_instance.get_spatial_placement(get_spatial_placement_request)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->get_spatial_placement: %s\n" % e)

Exception when calling DefaultApi->get_spatial_placement: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '72', 'Connection': 'keep-alive', 'cache-control': 'no-cache', 'content-security-policy': "base-uri 'self' cdn.humanatlas.io cdn.jsdelivr.net;script-src 'self' 'unsafe-inline' 'unsafe-eval' cdn.humanatlas.io cdn.jsdelivr.net unpkg.com www.googletagmanager.com;img-src 'self' 'unsafe-eval' cdn.humanatlas.io cdn.jsdelivr.net unpkg.com www.googletagmanager.com;connect-src *;default-src 'self';font-src 'self' https: data:;form-action 'self';frame-ancestors 'self';object-src 'none';script-src-attr 'none';style-src 'self' https: 'unsafe-inline';upgrade-insecure-requests", 'cross-origin-opener-policy': 'same-origin', 'cross-origin-resource-policy': 'same-origin', 'date': 'Tue, 09 Jul 2024 15:59:41 GMT', 'etag': 'W/"48-LlldxmjVeDK3aB9pU7mbZaega1c"', 'origin-agent-cluster': '?1', 'referrer-policy': 'no-referr

***
<a id='rui_loc'></a>
#### Get all hubmap rui locations (if enabled) &#x2705;
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [11]:
try:
    api_response = api_instance.gtex_rui_locations(cache = True)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

{'@context': 'https://hubmapconsortium.github.io/ccf-ontology/ccf-context.jsonld',
 '@graph': [{'@id': 'https://gtexportal.org/home/tissue/Artery_Coronary#FDonors',
             '@type': 'Donor',
             'consortium_name': 'GTEx',
             'description': 'Entered 1/26/2022, Kristin Ardlie, GTEx Project',
             'label': ['Females (n=94) Mean Age 53.5 (range 21 - 70)',
                       'Females (n=94), Mean Age 53.5 (range 21-70)'],
             'link': 'https://gtexportal.org/home/tissue/Artery_Coronary',
             'provider_name': 'GTEx',
             'provider_uuid': '083882bb-6cc6-4c12-a205-eac37c1a2640',
             'samples': [{'@id': 'https://gtexportal.org/home/tissue/Artery_Coronary#FDonors_TissueBlock1',
                          '@type': 'Sample',
                          'datasets': [{'@id': 'https://gtexportal.org/home/tissue/Artery_Coronary#FDonors_TissueBlock1_TissueBlock1_Dataset1',
                                        '@type': 'Dataset',
   

***
<a id='ontology_term'></a>
#### Get number of ontology term occurrences for a search &#x2705;
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional) 
</pre>

In [12]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.ontology_term_occurences(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

{'http://purl.obolibrary.org/obo/UBERON_0000955': 5,
 'http://purl.obolibrary.org/obo/UBERON_0002581': 5,
 'http://purl.obolibrary.org/obo/UBERON_0002661': 4,
 'http://purl.obolibrary.org/obo/UBERON_0002703': 5,
 'http://purl.obolibrary.org/obo/UBERON_0002813': 5,
 'http://purl.obolibrary.org/obo/UBERON_0006093': 1,
 'http://purl.obolibrary.org/obo/UBERON_0013702': 5,
 'http://purl.obolibrary.org/obo/UBERON_0035932': 4}


***
<a id='cell_term_occ'></a>
#### Get number of cell type term occurrences for a search &#x2705;
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional) 
</pre>

In [13]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.cell_type_term_occurences(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

{'http://purl.obolibrary.org/obo/CL_0000000': 5,
 'http://purl.obolibrary.org/obo/CL_0000057': 5,
 'http://purl.obolibrary.org/obo/CL_0000065': 5,
 'http://purl.obolibrary.org/obo/CL_0000084': 5,
 'http://purl.obolibrary.org/obo/CL_0000127': 5,
 'http://purl.obolibrary.org/obo/CL_0000128': 5,
 'http://purl.obolibrary.org/obo/CL_0000129': 5,
 'http://purl.obolibrary.org/obo/CL_0000236': 5,
 'http://purl.obolibrary.org/obo/CL_0000402': 5,
 'http://purl.obolibrary.org/obo/CL_0000540': 5,
 'http://purl.obolibrary.org/obo/CL_0000576': 5,
 'http://purl.obolibrary.org/obo/CL_0000623': 5,
 'http://purl.obolibrary.org/obo/CL_0000644': 5,
 'http://purl.obolibrary.org/obo/CL_0000679': 5,
 'http://purl.obolibrary.org/obo/CL_0000706': 5,
 'http://purl.obolibrary.org/obo/CL_0002453': 5,
 'http://purl.obolibrary.org/obo/CL_0002604': 5,
 'http://purl.obolibrary.org/obo/CL_0002605': 5,
 'http://purl.obolibrary.org/obo/CL_0010012': 5,
 'http://purl.obolibrary.org/obo/CL_0012000': 5,
 'http://purl.obolib

***
<a id='ontology_nodes'></a>
#### Get ontology term tree nodes &#x2705;
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [14]:
try:
    api_response = api_instance.ontology_tree_model(cache=True)
    for item in api_response:
        pprint(item)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

('root', 'http://purl.obolibrary.org/obo/UBERON_0013702')
('nodes',
 {'http://purl.obolibrary.org/obo/UBERON_0000002': OntologyTreeNode(id=None, label=None, synonym_labels=None, parent=None, children=None, additional_properties={'@id': 'http://purl.obolibrary.org/obo/UBERON_0000002', '@type': 'OntologyTreeNode', 'id': 'http://purl.obolibrary.org/obo/UBERON_0000002', 'parent': 'http://purl.obolibrary.org/obo/UBERON_0000995', 'children': ['http://purl.obolibrary.org/obo/UBERON_0013761', 'http://purl.obolibrary.org/obo/UBERON_0012247', 'https://purl.org/ccf/ASCTB-TEMP_cervical-mucosa', 'https://purl.org/ccf/ASCTB-TEMP_cervical-os', 'https://purl.org/ccf/ASCTB-TEMP_cervical-squamo-columnar-junction', 'http://purl.obolibrary.org/obo/UBERON_0004801', 'http://purl.obolibrary.org/obo/UBERON_0012249', 'http://purl.obolibrary.org/obo/UBERON_0000458'], 'synonymLabels': ['cervical canal of uterus', 'canalis cervicis uteri', 'caudal segment of uterus', 'cervix uteri', 'neck of uterus'], 'label': 'u

***
<a id='cell_nodes'></a>
#### Get cell type tree nodes &#x2705;
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [15]:
try:
    api_response = api_instance.cell_type_tree_model(cache = False)
    for item in api_response:
        pprint(item)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

('root', 'http://purl.obolibrary.org/obo/CL_0000000')
('nodes',
 {'http://purl.obolibrary.org/obo/CL_0000000': OntologyTreeNode(id=None, label=None, synonym_labels=None, parent=None, children=None, additional_properties={'@id': 'http://purl.obolibrary.org/obo/CL_0000000', '@type': 'OntologyTreeNode', 'id': 'http://purl.obolibrary.org/obo/CL_0000000', 'parent': '', 'children': ['http://purl.obolibrary.org/obo/CL_0000212', 'http://purl.obolibrary.org/obo/CL_0002071', 'http://purl.obolibrary.org/obo/CL_0000136', 'http://purl.obolibrary.org/obo/CL_0002521', 'http://purl.obolibrary.org/obo/CL_1000309', 'http://purl.obolibrary.org/obo/CL_0002619', 'http://purl.obolibrary.org/obo/CL_4028006', 'https://purl.org/ccf/ASCTB-TEMP_adventitial-stromal-cell', 'http://purl.obolibrary.org/obo/CL_1001096', 'http://purl.obolibrary.org/obo/CL_4033044', 'http://purl.obolibrary.org/obo/CL_4033055', 'http://purl.obolibrary.org/obo/CL_4033023', 'http://purl.obolibrary.org/obo/CL_4033024', 'http://purl.obolibr

***
<a id='providers'></a>
#### Get tissue provider names (for filtering) &#x2705;
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [16]:
try:
    api_response = api_instance.provider_names(cache = True)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

['Allen Institute',
 'Broad Institute',
 'CIR/UEdinburgh',
 'GTEx',
 'GUDMAP',
 'HTAN',
 'IsletCore',
 'KPMP-IU/OSU',
 'NHLBI/LungMap',
 'NIH',
 'RTI-General Electric',
 'TMC-CalTech',
 'TMC-Florida',
 'TMC-Stanford',
 'TMC-UCSD',
 'TMC-UConn',
 'TMC-Vanderbilt',
 'UCLA/SPARC',
 'UConn Health TMC',
 'UIowa',
 'UT Southwestern/GUDMAP',
 'University of California San Diego TMC',
 'University of Pittsburgh TMC',
 'Washington University TMC',
 'Wellcome Sanger Institute']


***
<a id='ref_organs'></a>
#### Get all reference organs &#x2705;
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [17]:
try:
    api_response = api_instance.reference_organs(cache = True)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

[SpatialEntity(label=None, comment=None, rui_rank=None, sex='Female', side=None, slice_count=None, slice_thickness=None, placement=None, object=None, color=None, entity_id=None, ccf_annotations=None, representation_of='http://purl.obolibrary.org/obo/UBERON_0004537', reference_organ='https://purl.humanatlas.io/ref-organ/blood-vasculature-female/v1.3#primary', creator=None, creator_first_name=None, creator_last_name=None, creator_orcid=None, creation_date=datetime.date(2024, 6, 15), x_dimension=192.7564, y_dimension=799.2176, z_dimension=184.9262, dimension_units='millimeter', additional_properties={'@id': 'https://purl.humanatlas.io/ref-organ/blood-vasculature-female/v1.3#primary', '@type': 'SpatialEntity', 'object': {'@id': 'https://purl.humanatlas.io/ref-organ/blood-vasculature-female/v1.3#primary_obj', '@type': 'SpatialObjectReference', 'file_format': 'model/gltf-binary', 'file_subpath': 'VH_F_blood_vasculature', 'file': 'https://cdn.humanatlas.io/digital-objects/ref-organ/blood-vasc

***
<a id='3d_scene'></a>
#### Get all nodes to form the 3D scene for an organ &#x2705;
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional)
    organ-iri - (string) (required)
</pre>

In [18]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
organ_iri = "http://purl.obolibrary.org/obo/UBERON_0004539"
token = ""

try:
    # Get aggregate results / statistics
    api_response = api_instance.reference_organ_scene(organ_iri=organ_iri, age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token
)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

[SpatialSceneNode(name=None, tooltip=None, unpickable=None, geometry=None, lighting=None, zoom_based_opacity=None, zoom_to_on_load=None, scenegraph=None, scenegraph_node=None, color=None, opacity=None, transform_matrix=None, priority=None, entity_id=None, ccf_annotations=None, representation_of='http://purl.obolibrary.org/obo/UBERON_0004539', reference_organ='https://purl.humanatlas.io/ref-organ/kidney-female-right/v1.3#primary', sex='Female', additional_properties={'@id': 'https://purl.humanatlas.io/ref-organ/kidney-female-right/v1.3#primary', '@type': 'SpatialSceneNode', 'tooltip': 'Right kidney', 'scenegraph': 'https://cdn.humanatlas.io/digital-objects/ref-organ/kidney-female-right/v1.3/assets/3d-vh-f-kidney-r.glb', 'scenegraphNode': 'VH_F_right_kidney', 'transformMatrix': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0.10872150000000001, -0.1985958, 0.1344381, 1], 'color': [255, 255, 255, 255], 'opacity': 0.2, 'unpickable': True, '_lighting': 'pbr'})]


***
<a id='nodes_3d'></a>
#### Get all nodes to form the 3D scene of reference body, organs, and tissues &#x2705;
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional)
</pre>

In [19]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    # Get aggregate results / statistics
    api_response = api_instance.scene(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token
)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

[SpatialSceneNode(name=None, tooltip=None, unpickable=None, geometry=None, lighting=None, zoom_based_opacity=None, zoom_to_on_load=None, scenegraph=None, scenegraph_node=None, color=None, opacity=None, transform_matrix=None, priority=None, entity_id=None, ccf_annotations=None, representation_of='http://purl.obolibrary.org/obo/UBERON_0002097', reference_organ='https://purl.humanatlas.io/ref-organ/skin-female/v1.4#primary', sex='Female', additional_properties={'@id': 'https://purl.humanatlas.io/ref-organ/skin-female/v1.4#primary', '@type': 'SpatialSceneNode', 'tooltip': 'skin', 'scenegraph': 'https://cdn.humanatlas.io/digital-objects/ref-organ/skin-female/v1.4/assets/3d-vh-f-skin.glb', 'scenegraphNode': 'VH_F_skin', 'transformMatrix': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], 'color': [255, 255, 255, 255], 'opacity': 0.5, 'unpickable': True, '_lighting': 'pbr', 'zoomBasedOpacity': False}),
 SpatialSceneNode(name=None, tooltip=None, unpickable=None, geometry=None, lighting=None, z

***
<a id='tech'></a>
#### Get technology names (for filtering) &#x2705;
<pre>
Accepted Parameters:
    cache - (boolean) (optional)
    token - (string) (optional)
</pre>

In [20]:
try:
    api_response = api_instance.technology_names(cache = False)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

['10x',
 'AF',
 'ATACseq',
 'CODEX',
 'Cell DIVE',
 'DESI',
 'H&E',
 'Histology',
 'IMC',
 'LC',
 'Light Sheet',
 'MALDI',
 'OTHER',
 'Publication',
 'RNAseq',
 'Slide-seq',
 'WGS',
 'seqFish',
 't-CyCIF']


***
<a id='tissue_block'></a>
#### Get Tissue Block Results &#x2705;
<pre>
Accepted Parameters:
    age - (MinMax) (optional) (min = 1.0; max = 110.0)
    bmi - (MinMax) (optional) (min = 13.0; max = 83.0)
    cache - (boolean)  (optional) (defaults to True)
    sex - (string)  (optional) (male, female, both)
    technologies - (string) (optional) (List of technologies - api_instance.technology_names)
    providers - (string) (optional) 
    ontology_terms - (string) (optional) 
    cell_type_terms - (string) (optional) 
    spatial - SpatialSearch(x, y, z, radius, target) (optional) 
    token - (string) (optional)
</pre>

In [21]:
age = MinMax(min=45.0, max=60.0)
bmi = MinMax(min=65.0, max=83.0)
cache = True
sex = "female"
technologies = []
providers = []
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0000955",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",] 
spatial = [SpatialSearch(
            x=84.0,
            y=146.0,
            z=53.0,
            radius=12.0,
            target="http://purl.org/ccf/latest/ccf.owl#VHFAllenBrain",
        ),]
token = ""

try:
    api_response = api_instance.tissue_blocks(age=age, bmi=bmi, cache=cache, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms, providers=providers, sex=sex, spatial=spatial, technologies=technologies, token=token
)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

[TissueBlock(sample_type='Tissue Block', spatial_entity_id=None, donor=None, section_count=None, section_size=None, section_units='millimeter', sections=None, label='Registered 7/28/2022, Raymond Sanchez, Allen Institute', description='5 x 5 x 10 millimeter, 10 millimeter', link='https://www.nature.com/articles/s41586-021-03465-8', datasets=[], additional_properties={'@id': 'https://api.cellxgene.cziscience.com/dp/v1/collections/367d95c0-0eb0-4dae-8276-9407239421ee#Donor1_TissueBlock2', '@type': 'Sample', 'donor': {'@id': 'https://api.cellxgene.cziscience.com/dp/v1/collections/367d95c0-0eb0-4dae-8276-9407239421ee#H18.30.001_Donor', '@type': 'Donor', 'link': 'https://www.nature.com/articles/s41586-021-03465-8', 'description': 'Entered 8/29/2022, Raymond Sanchez, Allen Institute for Brain Science', 'label': 'Female, Age 60', 'providerName': 'Allen Institute'}, 'sections': [{'@id': 'https://api.cellxgene.cziscience.com/dp/v1/collections/367d95c0-0eb0-4dae-8276-9407239421ee#Donor1_TissueBl

***
<a id='gtex_rui'></a>
#### Get all GTEx rui locations (if enabled) &#x2705;
<pre>
Accepted Parameters:
    cache - (boolean) (optional) 
</pre>

In [22]:
try: 
    api_response = api_instance.gtex_rui_locations(cache = False)
    pprint(api_response)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

{'@context': 'https://hubmapconsortium.github.io/ccf-ontology/ccf-context.jsonld',
 '@graph': [{'@id': 'https://gtexportal.org/home/tissue/Artery_Coronary#FDonors',
             '@type': 'Donor',
             'consortium_name': 'GTEx',
             'description': 'Entered 1/26/2022, Kristin Ardlie, GTEx Project',
             'label': ['Females (n=94) Mean Age 53.5 (range 21 - 70)',
                       'Females (n=94), Mean Age 53.5 (range 21-70)'],
             'link': 'https://gtexportal.org/home/tissue/Artery_Coronary',
             'provider_name': 'GTEx',
             'provider_uuid': '083882bb-6cc6-4c12-a205-eac37c1a2640',
             'samples': [{'@id': 'https://gtexportal.org/home/tissue/Artery_Coronary#FDonors_TissueBlock1',
                          '@type': 'Sample',
                          'datasets': [{'@id': 'https://gtexportal.org/home/tissue/Artery_Coronary#FDonors_TissueBlock1_TissueBlock1_Dataset1',
                                        '@type': 'Dataset',
   

***
<a id='token_usage'></a>
#### Token Usage &#x2705;

In [23]:
token = 'abbee123099245'
result = api_instance.db_status(token=token)
pprint(result)

DatabaseStatus(status='Ready', checkback=3600000, load_time=22594, message='Database successfully loaded')
